install pytorch-geometric
on mac: using pip wheel installation instructions, with python 3.9 and pytorch via conda (https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html)

loading data: 
ultimately: build on-disk dataset w/ pytorch 
atm: build dataset on the fly with synthetic dataset + data loader

In [1]:
import numpy as np
# coords used in segmentation paper (polar is 2d, how?)
#relative_polar = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_polar_3.npy')
#relative_coord = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_3.npy')
#edge_index_seg = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/edge_index_3.npy')

In [2]:
import importlib
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)

import logging
logging.basicConfig(level=logging.INFO)

Setting MELD_DATA_PATH to /Users/hannah.spitzer/projects/MELD/test_meld
Setting BASE_PATH to /Users/hannah.spitzer/projects/MELD/Data
Setting EXPERIMENT_PATH to /Users/hannah.spitzer/projects/MELD/experiments
Setting FS_SUBJECTS_PATH to /Users/hannah.spitzer/projects/MELD/test_meld/output/fs_outputs
No scripts_dir defined in /Users/hannah.spitzer/projects/MELD/meld_classifier/meld_classifier/meld_config.ini!


In [5]:

network_parameters = {
    'network_type': 'MoNet',
    'model_parameters': {
        'layer_sizes': [30,30,30],
        'dim': 2, # pseudo-coord dim
        'kernel_size': 3, # number of gaussian kernels
    },
    'training_parameters': {
        "max_patience": 400,
        "num_epochs": 800,
        'lr': 1e-2,
        'loss': 'cross_entropy',
        "batch_size": 1,
        "shuffle_each_epoch": True,
    },
    # experiment name. If none, experiment is not saved TODO implement
    'name': None,   #"date": datetime.datetime.now().strftime("%y-%m-%d"),
}

data_parameters = {
    'hdf5_file_root': "{site_code}_{group}_featurematrix.hdf5",
    'site_codes': ['H4'],
    'scanners': ['15T', '3T'],
    'dataset': 'MELD_dataset_V1.csv',
    'group': 'both',
    "features_to_exclude": [],
    "subject_features_to_exclude": [],
    "features": ['.on_lh.curv.mgh',
        '.on_lh.gm_FLAIR_0.25.mgh',
        '.on_lh.gm_FLAIR_0.5.mgh',
        '.on_lh.gm_FLAIR_0.75.mgh',
        '.on_lh.gm_FLAIR_0.mgh',
        '.on_lh.pial.K_filtered.sm20.mgh',
        '.on_lh.sulc.mgh',
        '.on_lh.thickness.mgh',
        '.on_lh.w-g.pct.mgh',
        '.on_lh.wm_FLAIR_0.5.mgh',
        '.on_lh.wm_FLAIR_1.mgh'],
    "features_to_replace_with_0": [], # specify this if manually specifying features
    "number_of_folds": 10,
    "fold_n": 0,
    "preprocessing_parameters": {
        "scaling": "scaling_params_GDL.json"
    },
    "icosphere_parameters": {
        "distance_type": "exact", #"exact",  # exact or pseudo
    },
    "combine_hemis": None,  # None, stack, TODO: combine with graph
}



In [7]:
exp = meld_graph.experiment.Experiment(network_parameters, data_parameters, save=False)
_ = exp.get_train_val_test_ids()
exp.data_parameters['train_ids'] = exp.data_parameters['train_ids'][:2]
exp.data_parameters['val_ids'] = exp.data_parameters['val_ids'][:2]
ds = meld_graph.dataset.GraphDataset.from_experiment(exp, mode='train')

INFO:meld_graph.dataset:Loading and preprocessing data
INFO:meld_graph.dataset:Combine hemis None
INFO:meld_graph.data_preprocessing:Scale data using file /Users/hannah.spitzer/projects/MELD/Data/scaling_params_GDL.json
INFO:meld_graph.data_preprocessing:Scale data using file /Users/hannah.spitzer/projects/MELD/Data/scaling_params_GDL.json


In [10]:
exp.load_model()

INFO:meld_graph.icospheres:Using coord type exact


In [11]:
edges = exp.model.icospheres.get_edges(level=1)

In [18]:
x = ds[0].x

In [19]:
x.shape

torch.Size([163842, 11])

In [54]:
#pool from 7 to 6
import torch
hex_5 = torch.LongTensor(np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/hexagons_6.npy'))

In [58]:
hex_5


tensor([[    0,     0, 18097,  ..., 19649, 23977, 23976],
        [    1,     1, 25168,  ..., 22409, 20448, 20449],
        [    2,     2, 21233,  ..., 22785, 28681, 28680],
        ...,
        [40959,  3969, 15744,  ..., 10241, 40960, 40961],
        [40960, 40961, 40959,  ..., 40947, 22031,  5576],
        [40961, 15745,  3969,  ..., 40960,  5576, 22032]])

In [34]:
neighbours = list(exp.model.icospheres.icospheres[6]['neighbours'])



In [41]:
neigh6 = np.concatenate([np.array(neighbours[:12])[:,0:1], np.array(neighbours[:12])], axis=1)
neigh6 = np.concatenate([neigh6, np.array(neighbours[12:])])


In [43]:
neigh6.shape

(40962, 6)

In [60]:
from torch import nn
class HexPool(nn.Module):
    def __init__(self, icoshperes, level):
        super(HexPool, self).__init__()
        self.neigh = icoshperes.get_neighbours(level=level)
        
    
    def forward(self, x):
        x = x[:len(self.neigh)][self.neigh]
        x = torch.max(x, dim=1)[0]
        return x

class HexUnpool(nn.Module):
    def __init__(self, icoshperes, level, device):
        super(HexUnpool, self).__init__()
        self.upsample = icoshperes.get_upsample(level=level)
        self.neigh = icoshperes.get_neighbours(level=level)
        self.device = device
        
    def forward(self, x):        
        limit = int(x.shape[0])
        new_x = torch.zeros(self.neigh.shape[0],x.shape[1]).to(self.device)
        new_x[:limit] = x
        new_x[limit:] = torch.mean(x[self.upsample],dim=1)
        return new_x




torch.Size([163842, 11])
(40962, 6)
torch.Size([40962, 6, 11])
torch.Size([40962, 11])


In [ ]:
neighbours

In [14]:
edges.shape

torch.Size([2, 282])

In [ ]:
exp.train()

/home/co-spit1/.conda/envs/meld_graph/lib/python3.9/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA A100-SXM-80GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA A100-SXM-80GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [11]:
from meld_classifier.meld_cohort import MeldCohort

cohort = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat6.hdf5', )

Init signature:
MeldCohort(
    hdf5_file_root='{site_code}_{group}_featurematrix_combat.hdf5',
    dataset=None,
    data_dir='',
)
Docstring:      Class to define cohort-level parameters such as subject ids, mesh
File:           ~/projects/MELD/meld_classifier/meld_classifier/meld_cohort.py
Type:           type
Subclasses:     
